In [1]:
import numpy as np

In [2]:
from src.solution_methods.linear_programming.cplex_dual_cmdp_solver import solve_CMDP_for_occupancy_measures
from src.concrete_decision_processes.maze_cmdp import RoseMazeCMDP

cmdp = RoseMazeCMDP()
constraint_vals, objective_value, occupancy_measures, variable_names = solve_CMDP_for_occupancy_measures(cmdp, True, False)

Entering cplex: view dual_mdp_result_20230307_102157.log for info
Exiting cplex


In [5]:
constraint_vals, objective_value

({'C_0': '0.314 L 0.314'}, 0.68767614)

In [4]:
occupancy_measures[0:5]

[0.0, 0.0, 0.0, 0.2826, 0.40507614]

In [5]:
variable_names[0:5]

['y(0, 0)', 'y(0, 1)', 'y(0, 2)', 'y(0, 3)', 'y(1, 0)']

In [31]:
variable_names_array = np.array(variable_names)
new_shape = (cmdp.n_states, cmdp.n_actions)
variable_names_mat = variable_names_array.reshape(new_shape)
assert variable_names_mat[2, 3] == 'y(2, 3)'
Q_sigma = np.array(occupancy_measures).reshape(new_shape)

In [32]:
A_star = [
    [an for an in range(cmdp.n_actions) if Q_sigma[sn, an] > 0]
    for sn in range(cmdp.n_states)
]
A_star = [
    action_list if len(action_list) > 0 else [-1]
    for action_list in A_star
]

In [33]:
np.random.choice(A_star[0])

3

In [34]:
phi_1 = {
    sn: np.random.choice(A_star[sn])
    for sn in range(cmdp.n_states)
}

In [35]:
phi_1

{0: 3, 1: 0, 2: 0, 3: 3, 4: 3, 5: -1, 6: 0, 7: 1, 8: 0}

In [ ]:
Q_phi_1 =